# 3.3 segment trees

In [1]:
MAX_N = 1 << 17

In [2]:
n_ = 5

In [3]:
n = 1
while n < n_: n*=2


In [4]:
import numpy as np

In [7]:
data = np.ones(2*n-1)*np.inf

In [8]:
def update(k: int, a: int):
    """k番目の値をa に変更
    """
    # 気の都合上、n-1 が一番下の層のスタートになる
    k += n - 1
    data[k] = a
    # 登りながら更新
    while k > 0:
        k = int((k-1)/2)
        data[k] = min(data[k*2+1], data[k*2+2])

In [10]:
def query(a: int, b:int, k: int, l:int, r: int):
    """ [a, b) の最小値を求める
    後ろの方の引数は、計算の簡単のための引数
    k は接点の番号, l, r, はその接点が [l, r) に対応することを表す
    したがって、外からは query(a, b, 0, 0, n) として呼ぶ
    """
    # [a, b) と [l, r) が交差しなければ inf
    if (r <= a or b <= l):
        return np.inf
    
    # [a, b) が [l, r) を完全に含んでいれば、この接点の値
    if a <= l and r <=b:
        return data[k]

    # そうでなければ、2つの子の最小値
    else:
        vl = query(a, b, k*2+1, 1, int((l+r)/2))
        vr = query(a, b, k*2+2, int((l+r)/2), r)
    

# クレーン

In [11]:
ST_SIZE = (1<<15)-1

In [17]:
MAX_N = 20
MAX_C = 20

In [14]:
vx = np.zeros(ST_SIZE)
vy = np.zeros(ST_SIZE)
ang = np.zeros(ST_SIZE)

In [18]:
prv = np.zeros(MAX_N)
L = np.zeros(MAX_N)
S = np.zeros(MAX_C)
A = np.zeros(MAX_N)

In [20]:
## segment tree
# k: 接点, l, r はその接点が [l, r) に対応することを表す
def init(k: int, l:int, r:int):
    """
    segment tree
    k: 接点, l, r はその接点が [l, r) に対応することを表す
    """
    
    ang[k] = vx[k] = 0
    # 葉
    # 葉であるならば、r と lがその接点の実際の数になっているため
    if (r - l) == 1:
        vy[k] = L[l]
    # 葉でない接点
    else:
        child_left = k*2 + 1
        child_right = k*2 + 2
        init(child_left, l, int((l+r)/2))
        init(child_right, int((l+r)/2), r)
        vy[k] = vy[child_left] + vy[child_right]

In [23]:
import math

In [22]:
def middle(l, r) -> int:
    """ 中間を返す
    """
    return int((l+r)/2)

In [24]:
def change(s: int, a: float, v: int, l: int, r: int):
    """ 場所sの角度がaだけ変更になった
    v は接点の番号、l, r はその接点が[l, r) に対応づいていることを表す
    """
    if s <= l:
        return
    if s >= r:
        return
    
    # l < s < r
    # 葉の時は何もしない!
    child_left = v*2 + 1
    child_right = v*2 + 2
    
    m = middle(l, r)
    change(s, a, child_left, l, m)
    change(s, a, child_right, m, r)
    
    if s <= m:
        ang[v] += a
    
    s = math.sin(ang[v])
    c = math.cos(ang[v])
    
    vx[v] = vx[child_left] + c * vx[child_right] - s * vy[child_right]
    vy[v] = vy[child_left] + s * vx[child_right] + c * vy[child_right]


In [25]:
def solve():
    init(0, 0, N)
    for i in range(1, N):
        prv[i] = math.pi
    
    # 各クエリを処理
    for i in range(C):
        s = S[i]
        a = A[i] / 360 * 2 * math.pi
        
        change(s, a-prv[s], 0, 0, N)
        
        print(f"{vx[0]:.2f} {vy[0]:.2f}")

In [27]:
N = 8

In [28]:
solve()

NameError: name 'C' is not defined

# Binary Indexed Tree

In [29]:
MAX_N = 100

In [30]:
import numpy as np

In [31]:
bit = np.zeros(MAX_N)

In [35]:
bin(3 & 1)

'0b1'

In [42]:
def add_two(a, b):
    print(f"{a:08b} + \n{b:08b} = \n{a+b:08b}")

In [44]:
add_two(1, 3)

00000001 + 
00000011 = 
00000100


In [47]:
i = 3

In [59]:
def reomve_last_bit(i):
    print(f"{i:08b}: i \n{-i:08b}: -i \n{i&-i:08b}: i&-i")

In [62]:
reomve_last_bit(10)

00001010: i 
-0001010: -i 
00000010: i&-i


In [76]:
def zero_comp(num, digits=4):
    assert num < 1 << digits, f"Can only take up to {1<<digits}, you passed {num}"
    return format(num % (1 << digits), f'0{digits}b')

In [84]:
def remove_last_bit(i, num_digits=4):
    print(f"i={i}")
    for exp in [i, -i, i&-i]:
        print(zero_comp(exp, num_digits))

In [86]:
remove_last_bit(20, 5)

i=20
10100
01100
00100


In [118]:
from typing import List
class BubbleSortCounter:
    
    def __init__(self, n: int, array: List):
        """ Make a bubble sort counter from binary indexed tree
        
        Args:
            n: length of list
            a: list to sort
        
        Attributes:
            bit: The numbers to keep track of
                The sum of the leaves of [0-a_i]
        """
        self.n = n
        self.array = array
        # [1, n] の数を利用したい
        self.bit = np.zeros(n+1)
    
    def add(self, index: int, value: float):
        """Add `value` to bit[`index`] and update the leaves
        
        Args:
            index(int): 
            value(float):
        """
        while index <= self.n:
            self.bit[index] += value
            # get the lowest bit of index, and add
            # this will traverse up the binary tree, 5(0101)-> 6(0110) -> 8(1000)
            index += index & -index
    
    def reduce_sum(self, index: int):
        """Reduct the tree into one sum
            This will add the sum of bit[0:index]
        
        Args:
            index(int): last index of the range to sum
        """
        sum_adder = 0
        while index:
            sum_adder += self.bit[index]
            # remove the last bit
            # this will travese the tree downwards, from 5(0101)->4(0100)->0
            index -= index & -index
        return sum_adder

    def solve(self):
        """Solve the bubble sort problem
        """
        ans = 0
        for j in range(self.n):
            ans += j - self.reduce_sum(self.array[j])
            self.add(self.array[j], 1)
        return ans
            
        

In [119]:
BubbleSortCounter(n, a).solve()

3.0

In [110]:
n = 4
a = [3, 1, 4, 2]

In [ ]:
def add(i, x):
    while i <= n:
        bit[i] += x
        i += i & -i

In [ ]:
def sum(i: int):
    s = 0
    while i:
        s += bit[i]
        i -= i & -i
    return s

In [98]:
def solve():
    ans = 0
    for j in range(n):
        ans += j - sum(a[j])
        add(a[j], 1)
    return ans

In [99]:
solve()

-17.0